In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import keras
from keras import losses
from keras import metrics
from keras import optimizers

Using TensorFlow backend.


In [43]:
dfx = pd.read_csv(
  "FF.csv",
  parse_dates=[0],
  index_col= 0,
  header = None
)


In [26]:
dfx.head()

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9
2018-04-15,38.0,12.8,14.0,33.0,13.0,31.0,0.4,11.0,0.0,264
2018-04-15,38.0,12.8,14.0,33.0,13.0,31.0,0.4,11.0,0.0,264
2018-04-16,40.0,12.8,13.0,35.0,5.0,28.0,0.0,7.0,0.0,264
2018-04-16,40.0,12.8,13.0,35.0,5.0,28.0,0.0,7.0,0.0,264


In [70]:
def traintest(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
    return np.array(Xs), np.array(ys)

In [68]:
def createDataframe():
    df = dfx.iloc[-50000:]
    train_size = int(len(df) * 0.8)
    test_size = len(df) - train_size
    train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
    
    from sklearn.preprocessing import RobustScaler
    time_steps = 10
    
    f_columns = range(1,13)
    
    f_transformer = RobustScaler()
    cnt_transformer = RobustScaler()

    f_transformer = f_transformer.fit(train[f_columns].to_numpy())
    cnt_transformer = cnt_transformer.fit(train[[1]])

    train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
    train[1] = cnt_transformer.transform(train[[1]])

    test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
    test[1] = cnt_transformer.transform(test[[1]])
    
    
    X_train, y_train = traintest(train, train[1], time_steps)
    X_test, y_test = traintest(test, test[1], time_steps)

    return [X_test, cnt_transformer]
    

In [41]:
def init():
    from keras.models import load_model
    global model
    model = tf.keras.models.load_model('outputs/models/LSTMTest.h5')
  
    model.compile('adam','mse')

In [58]:
def run(input_np, cnt_transformer):
    import json
    
    from sklearn.preprocessing import RobustScaler
    
    pred = model.predict(input_np)
    pred = cnt_transformer.inverse_transform(pred.reshape(1, -1))
    pred = cnt_transformer.inverse_transform(pred)
    return json.dumps(str(pred))

In [71]:
X_test, scaler = createDataframe()
init()
run(X_test, scaler)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

'"[[-3.0076537 -3.0068595 -3.102638  ... -3.322866  -3.3676353 -3.3649843]]"'